In [ ]:
# Generate a Utilities Usage report for a given unit

# Set dates in YYYY-MM-DD format
start_date = '2020-01-01'  
end_date = '2020-12-31'

# Specify where to find the input files
# the root directory where you are running jupyter notebook

# Transactions is CSV file that tracks all property related credits and debits
# move your MINT.com transactions.csv into this project directory
PATH_TO_YOUR_TRANSACTIONS = "transactions.csv"

# UTILS is a csv file that specifies the monthly utilities cost included in the rent for each unit
PATH_TO_UTIL_LIMITS = "util-limits.csv" # Used to calculate tenant charges for utilities

# Load Transaction Data from a CSV File

import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick


# Set the name of the column where the transactions are categorized
# In Mint this is usually "Category".  I use Mint's category to link 
# transactions to a property and the add a seperate "Label" column
# after exporting the data for a property from Mint
CATEGORY = "Label"

# Load the transaction data from the csv into a dataframe
parse_dates = ['Date']
df = pd.read_csv(PATH_TO_YOUR_TRANSACTIONS, parse_dates=parse_dates)
df.set_index(['Date'], inplace=True)
df['Amount'] = df['Amount'].astype(float)

# Get rid of Mint columns that we don't care about
df.drop(['Category', 'Original Description','Account Name', 'Notes'], axis=1, inplace=True)

if df.empty:
    print('No data to process for date range ', start_date, '-', end_date)
else:
    print('Will analyze utilities transactions from', start_date, 'to', end_date)


In [ ]:
# Sanity check that all entries that should have a unit do
unit_entries = df[(df[CATEGORY] == 'Utilities') | (df[CATEGORY] == 'Rent')]

if not unit_entries[unit_entries['Unit'].isnull()].empty:
    unit_entries[unit_entries['Unit'].isnull()]
else:
    print('Transaction data includes unit info for all transactions in the rent and utilities categories.')

In [ ]:
# Read in the monthly utilities max before reimbursement.
# This cell has no output
limit_df = pd.read_csv(PATH_TO_UTIL_LIMITS, parse_dates=False)
limit_df.set_index(['Unit'], inplace=True)
limit_df['Amount'] = limit_df['Amount'].astype(float)


In [ ]:
# Remove any non-utility related transactions 

df = df[df[CATEGORY] == 'Utilities']
df.sort_index(inplace=True)
# Remove transactions outside of the specified date range
df = df.loc[start_date:end_date]

if not df.empty:
    print('Will analyze %d utilities transaction in the time period' % len(df))
else:
    print('No utilities data for the date range and unit specified.')


In [ ]:
# Function to generate the report for a unit
# This cell does not generate output but is needed by the next cell

def generate_unit_report(UNIT, df):
    # Remove transactions related to other units
    unit_df = df[df['Unit'] == UNIT]
    # Drop those columns
    unit_df.drop([CATEGORY,'Unit'], axis=1, inplace=True)
    # Get the limit for this unit
    UNIT_UTIL_LIMIT = limit_df.loc[UNIT].Amount
    if not UNIT_UTIL_LIMIT:
        print('Cannot find the allowed utilities expense included in rent for unit:', UNIT)

    print('Utilities Usage Report for 122 Spring St, Apt %s' % UNIT)
    print('Report Period:', start_date, '-', end_date)
    print('Monthly Utilities included in rent:${:,.2f}'.format(UNIT_UTIL_LIMIT))

    usage = unit_df[unit_df['Transaction Type'] == 'debit']
    credits = unit_df[unit_df['Transaction Type'] == 'credit']
    payments_received = 0.0

    if not credits.empty:
        print('\nReimbursement payments recieved:')
        payments_received = credits.Amount.sum();
        display(credits[['Description', 'Amount']])


    if not usage.empty:
        # Calculate any months that incurred overages
        overage = usage.assign(Overage=lambda x: x.Amount - UNIT_UTIL_LIMIT)
        overage.Overage[overage.Overage < 0] = 0

        # Display table monthly charges with overage info
        print('\nUtility payments made by landlord during period:')
        display(overage[['Description', 'Amount', 'Overage']])

        if overage.Overage.sum() > 0:
            print('Overage for report period: ${:,.2f}'.format(overage['Overage'].sum()))
            if payments_received > 0:
                print('Payments received in period: ${:,.2f}'.format(payments_received), '\n')
                print('Outstanding balance due: ${:,.2f}'.format(overage['Overage'].sum() - payments_received))
                

    else:
        print('No Utilities usage for period.')
        
    print('\n\n')



In [ ]:
# Loop through all the units and print a report

# Disable warning generated when setting negative overage values to zero
pd.set_option('mode.chained_assignment', None)

for UNIT in sorted(df['Unit'].unique()):
    generate_unit_report(UNIT, df)
# Renable warning        
pd.set_option('mode.chained_assignment', 'warn')
